In [1]:
import pandas as pd
import scrapy
from selenium import webdriver
import time
from datetime import datetime as dt

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd

class GoodReadsSpider(scrapy.Spider):

    name = "goodreads"

    def __init__(self):
        self.df = pd.DataFrame(columns=["quote_text", "author", "tags", "likes"])

    # requests
    def start_requests(self):
        url = "https://www.goodreads.com/quotes?page=1"
        yield scrapy.Request(url, callback=self.parse) # callback caches the response from each url
    
    # response
    def parse(self, response):
        for texts in response.xpath("//div[@class='quote']"):
            data = {
                "quote_text": texts.xpath(".//div/div/text()").get().strip().replace("”", "").replace("“", ""),
                "author": texts.xpath(".//div/div/span/text()").get().strip(),
                "tags": texts.xpath(".//div/div/div/a/text()").getall()[:-1], # excluding "likes"
                "likes": texts.xpath(".//div/div/div[@class='right']/a/text()").get().strip().replace("”", "").replace("“", "")             
            }

            self.df = self.df.append(data, ignore_index=True)
            yield data
        
    
    def to_dataframe(self, response):
        return pd.DataFrame(list(self.parse(response)))

    

process = CrawlerProcess()
process.crawl(GoodReadsSpider)
process.start()
process.join()


goodreads = GoodReadsSpider()
# response = process.spider_loader.load("goodreads").start_requests().next().callback
# df = goodreads.to_dataframe(response)
goodreads.df

2023-05-16 20:23:13 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-05-16 20:23:13 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.22621-SP0
2023-05-16 20:23:13 [scrapy.crawler] INFO: Overridden settings:
{}
2023-05-16 20:23:13 [py.warnings] WARNING: c:\Users\abume\anaconda3\lib\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST

,quote_text,author,tags,likes


In [54]:
# print(goodreads.df)